In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

### vvt 파일 불러오기

In [ ]:
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/Ben Platt Live From Radio City Music Hall/Ben.Platt.Live.from.Radio.City.Music.Hall.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/Inception/Inception.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/Klaus/Klaus.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/Monty Python And The Holy Grail/Monty.Python.and.the.Holy.Grail.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/The Boy In The Striped Pajamas/The.Boy.in.the.Striped.Pajamas.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/The Departed/The.Departed.WEBRip.Netflix.en[cc].vtt
# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/movies/There Will Be Blood/There.Will.Be.Blood.WEBRip.Netflix.en[cc].vtt

# /content/drive/MyDrive/elice_데이터분석_프로젝트/data/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.vtt

In [5]:
#Drama
Drama = ['Breaking Bad', 'Friends', 'Game of Thrones', 'Greys Anatomy', 'Modern Family', 'Sherlock', 'The Big Bang Theory', 'The Good Place']
Movie = ['Ben Platt Live From Radio City Music Hall', 'Inception', 'Klaus', 'Men In Black', 'Moneyball', 'Monty Python And The Holy Grail', 'The Boy In The Striped Pajamas', 'The Departed', 'The Intern', 'There Will Be Blood', 'Zootopia']

## 여기서 작품 선택합니다 꼭!!

In [7]:
title = Movie[0] # 여기서 작품 선택

if title in Drama:
    path = '../Script/Drama'
    file_name = title.replace(' ', '.') + '.WEBRip.Netflix.en[cc]'
    # file_name = 'Ben.Platt.Live.from.Radio.City.Music.Hall.WEBRip.Netflix.en[cc]'
else:
    path = '../Script/Movie'
    file_name = title.replace(' ', '.') + '.WEBRip.Netflix.en[cc]'
    # file_name = 'There.Will.Be.Blood.WEBRip.Netflix.en[cc]'
    
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [9]:
segments[:10]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2020/03/19 22:37:35',
 'NOTE SegmentIndex\nNOTE Segment=598.181 26242@794 148\nNOTE Segment=596.304 28310@27036 151\nNOTE Segment=581.915 34318@55346 181\nNOTE Segment=595.553 28527@89664 155\nNOTE Segment=593.301 25508@118191 137\nNOTE Segment=598.098 31147@143699 161\nNOTE Segment=595.512 27694@174846 153\nNOTE Segment=596.554 29109@202540 156\nNOTE Segment=348.557 11308@231649 64\nNOTE /SegmentIndex',
 '                                                                                                                                                                                                                                                                                                                                          ',
 '\n1\n00:00:05.839 --> 00:00:07.841 position:50.00%,middle align:middle size:80.00% line:84.67% \n<c.bg_transparent>[cars and buses passing by]</c.bg_transparent>',
 '2\n00:00:10.760 --

### 불필요한 부분 제거

In [10]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [11]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:20]

[None,
 None,
 None,
 None,
 None,
 '6\n00:00:28.153  00:00:29.529 And relax.',
 '7\n00:00:31.656  00:00:32.824 This is perfect here.',
 '8\n00:00:33.199  00:00:35.160 Yes, and we have one exercise to do.',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [14]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = split_segment[3:]
      times.append(script)
      trimmed_segments.append(times)

In [15]:
trimmed_segments[:10]

[['6', '00:00:28.153', '00:00:29.529', ['And', 'relax.']],
 ['7', '00:00:31.656', '00:00:32.824', ['This', 'is', 'perfect', 'here.']],
 ['8',
  '00:00:33.199',
  '00:00:35.160',
  ['Yes,', 'and', 'we', 'have', 'one', 'exercise', 'to', 'do.']],
 ['68', '00:05:18.151', '00:05:20.403', ['Hello,', 'Radio', 'City!']],
 ['69',
  '00:05:23.281',
  '00:05:24.907',
  ['Thank', 'you', 'so', 'much', 'for', 'being', 'here!']],
 ['70',
  '00:05:26.034',
  '00:05:28.202',
  ['If', 'you', 'happen', 'to', 'know', 'it,', 'feel', 'free', 'to', 'sing.']],
 ['127',
  '00:08:52.323',
  '00:08:54.325',
  ['Well', 'hello,', 'New', 'York', 'City,', 'how', 'are', 'you?']],
 ['128',
  '00:08:57.578',
  '00:09:00.706',
  ['I',
   'feel',
   'very',
   'overwhelmed',
   'because',
   'this',
   'is',
   'Radio',
   'City',
   'Music',
   'Hall.']],
 ['129',
  '00:09:03.000',
  '00:09:05.503',
  ['Yes,', "you've", 'heard', 'as', 'well,', 'I', 'see.']],
 ['130',
  '00:09:05.586',
  '00:09:07.672',
  ['Um,', 'and', 

### 데이터 저장

In [16]:
with open(str(file_name)[:-3]+'csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

### 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [17]:
# colab
data = pd.read_csv(file_name[:-3]+'csv', names=['num', 'start', 'end', 'script'])

# jupyter
# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv', names=['num', 'start', 'end', 'script'])

data

,num,start,end,script
0,6,00:00:28.153,00:00:29.529,"['And', 'relax.']"
1,7,00:00:31.656,00:00:32.824,"['This', 'is', 'perfect', 'here.']"
2,8,00:00:33.199,00:00:35.160,"['Yes,', 'and', 'we', 'have', 'one', 'exercise..."
3,68,00:05:18.151,00:05:20.403,"['Hello,', 'Radio', 'City!']"
4,69,00:05:23.281,00:05:24.907,"['Thank', 'you', 'so', 'much', 'for', 'being',..."
...,...,...,...,...
394,1247,01:19:26.386,01:19:27.471,"['and', 'with', 'their', 'love.']"
395,1248,01:19:28.138,01:19:29.514,"['So,', 'this', 'is', 'inspired', 'by', 'them.']"
396,1249,01:19:30.390,01:19:31.725,"['Thank', 'you', 'again', 'for', 'having', 'me.']"
397,1250,01:19:31.808,01:19:33.018,"['This', 'has', 'been', 'so', 'wonderful.']"


In [18]:
# colab
data.to_csv(file_name[:-3]+'csv', columns=['num', 'start', 'end', 'script'], index=False)
data = pd.read_csv(file_name[:-3]+'csv')

# jupyter
# data.to_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv', columns=['num', 'start', 'end', 'script'], index=False)
# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')

data

,num,start,end,script
0,6,00:00:28.153,00:00:29.529,"['And', 'relax.']"
1,7,00:00:31.656,00:00:32.824,"['This', 'is', 'perfect', 'here.']"
2,8,00:00:33.199,00:00:35.160,"['Yes,', 'and', 'we', 'have', 'one', 'exercise..."
3,68,00:05:18.151,00:05:20.403,"['Hello,', 'Radio', 'City!']"
4,69,00:05:23.281,00:05:24.907,"['Thank', 'you', 'so', 'much', 'for', 'being',..."
...,...,...,...,...
394,1247,01:19:26.386,01:19:27.471,"['and', 'with', 'their', 'love.']"
395,1248,01:19:28.138,01:19:29.514,"['So,', 'this', 'is', 'inspired', 'by', 'them.']"
396,1249,01:19:30.390,01:19:31.725,"['Thank', 'you', 'again', 'for', 'having', 'me.']"
397,1250,01:19:31.808,01:19:33.018,"['This', 'has', 'been', 'so', 'wonderful.']"


In [ ]:
# colab
# data = pd.read_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv')
# data = data.rename(columns = {'[]':'script'})
# data = data.rename(columns = {'Unnamed: 0':'num', 'Unnamed: 1':'start', 'Unnamed: 2':'end', '[]':'script'})
# data.to_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv')

# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')
# data = data.rename(columns = {'Unnamed: 0':'num', 'Unnamed: 1':'start', 'Unnamed: 2':'end', '[]':'script'})
# data.to_csv('Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')

### 파일 불러올때 불필요한 컬럼이 포함되기 때문에 index_col=0 포함해서 불러와야 합니다.

In [ ]:
# colab
# data = pd.read_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv', index_col=0)
# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')
# data